## This notebook used for evaluation of support vector machine model 

### includes

In [19]:

import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, classification_report, roc_curve, confusion_matrix,accuracy_score
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import seaborn as sn

### loading dataset

In [13]:
df=pd.read_csv('../dataset/diabetes_dataset.csv')
df.head()


,year,gender,age,location,race:AfricanAmerican,race:Asian,race:Caucasian,race:Hispanic,race:Other,hypertension,heart_disease,smoking_history,bmi,hbA1c_level,blood_glucose_level,diabetes
0,2020,Female,32.0,Alabama,0,0,0,0,1,0,0,never,27.32,5.0,100,0
1,2015,Female,29.0,Alabama,0,1,0,0,0,0,0,never,19.95,5.0,90,0
2,2015,Male,18.0,Alabama,0,0,0,0,1,0,0,never,23.76,4.8,160,0
3,2015,Male,41.0,Alabama,0,0,1,0,0,0,0,never,27.32,4.0,159,0
4,2016,Female,52.0,Alabama,1,0,0,0,0,0,0,never,23.75,6.5,90,0


In [14]:
df.describe

<bound method NDFrame.describe of        year  gender   age location  race:AfricanAmerican  race:Asian  \
0      2020  Female  32.0  Alabama                     0           0   
1      2015  Female  29.0  Alabama                     0           1   
2      2015    Male  18.0  Alabama                     0           0   
3      2015    Male  41.0  Alabama                     0           0   
4      2016  Female  52.0  Alabama                     1           0   
...     ...     ...   ...      ...                   ...         ...   
99995  2018  Female  33.0  Wyoming                     0           0   
99996  2016  Female  80.0  Wyoming                     0           1   
99997  2018    Male  46.0  Wyoming                     0           1   
99998  2018  Female  51.0  Wyoming                     1           0   
99999  2016    Male  13.0  Wyoming                     0           0   

       race:Caucasian  race:Hispanic  race:Other  hypertension  heart_disease  \
0                   

In [15]:
df.columns

Index(['year', 'gender', 'age', 'location', 'race:AfricanAmerican',
       'race:Asian', 'race:Caucasian', 'race:Hispanic', 'race:Other',
       'hypertension', 'heart_disease', 'smoking_history', 'bmi',
       'hbA1c_level', 'blood_glucose_level', 'diabetes'],
      dtype='object')

In [16]:
 
df.shape

(100000, 16)

In [ ]:
## we drop the unnessessary information
df.drop(columns=['year','location','race:AfricanAmerican','race:Asian','race:Caucasian','race:Hispanic','race:Other'],inplace=True)
df

,gender,age,hypertension,heart_disease,smoking_history,bmi,hbA1c_level,blood_glucose_level,diabetes
0,Female,32.0,0,0,never,27.32,5.0,100,0
1,Female,29.0,0,0,never,19.95,5.0,90,0
2,Male,18.0,0,0,never,23.76,4.8,160,0
3,Male,41.0,0,0,never,27.32,4.0,159,0
4,Female,52.0,0,0,never,23.75,6.5,90,0
...,...,...,...,...,...,...,...,...,...
99995,Female,33.0,0,0,never,21.21,6.5,90,0
99996,Female,80.0,0,0,No Info,36.66,5.7,100,0
99997,Male,46.0,0,0,ever,36.12,6.2,158,0
99998,Female,51.0,0,0,not current,29.29,6.0,155,0


In [24]:
encoder = LabelEncoder()
df['smoking_history'] = encoder.fit_transform(df['smoking_history'])
df['gender'] = encoder.fit_transform(df['gender'])
df['gender']

0        0
1        0
2        1
3        1
4        0
        ..
99995    0
99996    0
99997    1
99998    0
99999    1
Name: gender, Length: 100000, dtype: int64

### Data Preprocessing

In [6]:
# Shows how many nulls are in each column
print(df.isnull().sum())

year                    0
gender                  0
age                     0
location                0
race:AfricanAmerican    0
race:Asian              0
race:Caucasian          0
race:Hispanic           0
race:Other              0
hypertension            0
heart_disease           0
smoking_history         0
bmi                     0
hbA1c_level             0
blood_glucose_level     0
diabetes                0
dtype: int64
